In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

# Application of quantile-regression-based machine learning models for simulating blast movement

# Vol. 1. Data gathering cleaning and tidying

## Introduction

As a continuation of my Data Sceience project, this Notebook series aim to continue the previously established work and establish a regression-based simulation model. For the purpose differnet monitoring locations from different blast panels and their corresponding local conditions were gathered so as to provide enough useful data for a regressor to predict the 3D movement in an arbitrary monitoring location of a blast. The purpsoe of this regression model is to provide the foundation to quickly simulate and predict blast outcomes in order to adjust explsoive charging strategies and provide information on the potential location of the ore zones in a post-blast environment.

## Raw BMM spreadsheet cleaning

The movement report data was is generated through a specialized software (BMM Explorer) and the data is stored in an .xlsx format.

In [2]:
bmm_raw = pd.read_excel('All_Movement_Report_20210728.xlsx')
bmm_raw.iloc[:,29:].head()

,Post BMM RL,Det Depth,3D Dist.,Horiz. Dist.,Vert. Dist.,Direction,Inclination,Heave
0,478.804813,2.181187,2.614737,2.526958,0.671813,200.234937,14.888182,1.053
1,476.487346,3.777654,1.248611,1.246610,-0.070654,178.207218,-3.243882,0.107
2,478.519509,1.952491,2.352037,2.346485,0.161509,190.658654,3.937474,0.314
3,475.879929,5.466071,1.652971,1.597913,-0.423071,195.201656,-14.829647,1.343
4,478.638902,2.746098,1.897213,1.846457,0.435902,193.690050,13.282901,1.382


The provided data shows the post-blast-movement relative level, the detection depth of the monitor, the 3D distance of movement, the horizontal and vertical movement components, the direction and inclination of movement, as well as the heave of the blast (the difference between the pre-blast and post-blast surface elevation directly above the monitor). 

A further inspection of the model shows the number of observations and features in the dataset.

In [3]:
print(bmm_raw.shape)

(1689, 37)


In [4]:
bmm_raw.columns

Index(['Date', 'Location', 'Blast', 'Bench Height', 'Hole Diam.', 'Burden',
       'Spacing', 'No. of Holes', 'Hole Config', 'Confinement', 'Pattern Type',
       'Hole Depth', 'Stemming Length', 'Powder Factor', 'IR Time', 'IH Time',
       'Initiation Type', 'Explosive Type', 'Rock Type', 'BMM#', 'BMM Colour',
       'Collar East', 'Collar North', 'Collar RL', 'Pre BMM RL', 'Inst Depth',
       'After East', 'After North', 'Surface RL', 'Post BMM RL', 'Det Depth',
       '3D Dist.', 'Horiz. Dist.', 'Vert. Dist.', 'Direction', 'Inclination',
       'Heave'],
      dtype='object')

A list of all features of interest is made and the dataframed is filtered by them, as most of them are redundant for the analysis.

In [5]:
bmm_raw_features = [
    'Blast',
    'Hole Config',
    'Confinement',
    'Pattern Type',
    'Explosive Type',
    'Collar East', 
    'Collar North', 
    'Collar RL', 
    'Pre BMM RL', 
    'Inst Depth',
    'After East', 
    'After North', 
    'Surface RL', 
    'Post BMM RL', 
    'Det Depth',
    '3D Dist.', 
    'Horiz. Dist.', 
    'Vert. Dist.', 
    'Direction', 
    'Inclination',
    'Heave'
]

bmm_filtered = bmm_raw[bmm_raw_features].copy()

Additional features are estimated, including:
- Target surface - Design elevation of post-blast surface after the rock is excavated
- Bench height at monitor location (m)
- Installation height of monitor (m) - Distance from Target surface to monitor pre-blasting location
- Detection height of monitor (m) - Distance from Target surface to monitor post-blast location

In addition certain string manipulations are done to reduce 'Confinement' (indicates whether there is a buffer in front of the free face of the blast) values to lowercase strings.

Duplicate values are also removed from the dataset.

In [6]:
# Estimate additional parameters needed for data cleaning
bmm_filtered['Target Surface'] = (round(bmm_filtered['Collar RL']/5)-1)*5
bmm_filtered['Bench Height'] = bmm_filtered['Collar RL'] - bmm_filtered['Target Surface']
bmm_filtered['Install Height'] = bmm_filtered['Pre BMM RL'] - bmm_filtered['Target Surface']
bmm_filtered['Det Height'] = bmm_filtered['Post BMM RL'] - bmm_filtered['Target Surface']

# Convert uppercase letters to lowercase
bmm_filtered['Confinement'] = bmm_filtered['Confinement'].apply(lambda x: x.lower() if isinstance(x, str) else x)

# Drop duplicated samples
duplicates = bmm_filtered.duplicated(subset=['Collar East', 'Collar North', 'Pre BMM RL'])
bmm_filtered = bmm_filtered.drop_duplicates(subset=['Collar East', 'Collar North', 'Pre BMM RL'], keep='first')

bmm_filtered.iloc[:,13:].head()

,Post BMM RL,Det Depth,3D Dist.,Horiz. Dist.,Vert. Dist.,Direction,Inclination,Heave,Target Surface,Bench Height,Install Height,Det Height
0,478.804813,2.181187,2.614737,2.526958,0.671813,200.234937,14.888182,1.053,475.0,4.933,3.133,3.804813
1,476.487346,3.777654,1.248611,1.246610,-0.070654,178.207218,-3.243882,0.107,475.0,5.158,1.558,1.487346
2,478.519509,1.952491,2.352037,2.346485,0.161509,190.658654,3.937474,0.314,475.0,5.158,3.358,3.519509
3,475.879929,5.466071,1.652971,1.597913,-0.423071,195.201656,-14.829647,1.343,475.0,5.003,1.303,0.879929
4,478.638902,2.746098,1.897213,1.846457,0.435902,193.690050,13.282901,1.382,475.0,5.003,3.203,3.638902


### Removing outliers and atypical blasting conditions

The cosnideration of otliers for blast movement can be tricky, however, there are certain values which are more than obvious that are physically impossible to occur (e.g. the monitor is under the post-blast design surface or above the the surface of the muckpile). Additionally, certain magnitudes of movement which are rare and occur under very special conditions (even if they are not identified) are disregarded as outliers.

The outliers considered for this study fall into either one of the following onditions:
1. Pre-blast BMM elevation is above the target surface
2. Pre-blast BMM elevation is below the blast panel surface
3. Post-blast BMM elevation is below the muckpile surface
4. Post-blast BMM elevation is above the target surface
5. Square hole patterns are disregarded (only staggered patterns are analyzed)
6. Bench heights over 6m and below 4.25m are disregarded from the analysis
7. Vertical movement above 2m are treated as outleirs

In [7]:
conditions = (bmm_filtered['Install Height'] > 0) \
            & (bmm_filtered['Collar RL'] > bmm_filtered['Pre BMM RL']) \
            & (bmm_filtered['Post BMM RL'] < bmm_filtered['Surface RL']) \
            & (bmm_filtered['Install Height'] + bmm_filtered['Vert. Dist.'] >= 0) \
            & (bmm_filtered['Hole Config'] != 'Square' ) \
            & (bmm_filtered['Bench Height'] <= 6 )  & (bmm_filtered['Bench Height'] >= 4.25 ) \
            & (bmm_filtered['Vert. Dist.'] <= 2 ) 

df = bmm_filtered[conditions].reset_index(drop=True)
df.iloc[:,13:]

,Post BMM RL,Det Depth,3D Dist.,Horiz. Dist.,Vert. Dist.,Direction,Inclination,Heave,Target Surface,Bench Height,Install Height,Det Height
0,476.639344,4.711656,1.214168,1.213429,0.042344,207.430863,1.998595,1.254,475.0,5.097,1.597,1.639344
1,478.750420,2.950580,2.296334,2.251125,0.453420,209.514424,11.388110,1.604,475.0,5.097,3.297,3.750420
2,476.322787,4.532213,0.657434,0.656040,0.042787,210.199858,3.731591,0.875,475.0,4.980,1.280,1.322787
3,478.823708,2.165292,2.016424,1.967000,0.443708,198.895516,12.711780,1.009,475.0,4.980,3.380,3.823708
4,476.526922,3.996078,0.857800,0.857123,-0.034078,199.491946,-2.276801,0.262,475.0,5.261,1.561,1.526922
...,...,...,...,...,...,...,...,...,...,...,...,...
1398,443.566240,2.042760,0.470029,0.156416,0.443240,187.715674,70.562429,0.886,440.0,4.723,3.123,3.566240
1399,441.332644,4.447356,0.756384,0.745875,0.125644,162.925509,9.561801,0.873,440.0,4.907,1.207,1.332644
1400,444.018862,1.892138,0.766121,0.283185,0.711862,123.185025,68.306860,1.004,440.0,4.907,3.307,4.018862
1401,441.178918,4.593082,0.387790,0.376205,-0.094082,225.646166,-14.040684,1.099,440.0,4.673,1.273,1.178918


### Drop unnecessary columns and rename remaining ones for ease of use

For the sake of simplicity, all features considered to be crucial for analysis are renamed as shown bellow.

In [8]:
df.columns

Index(['Blast', 'Hole Config', 'Confinement', 'Pattern Type', 'Explosive Type',
       'Collar East', 'Collar North', 'Collar RL', 'Pre BMM RL', 'Inst Depth',
       'After East', 'After North', 'Surface RL', 'Post BMM RL', 'Det Depth',
       '3D Dist.', 'Horiz. Dist.', 'Vert. Dist.', 'Direction', 'Inclination',
       'Heave', 'Target Surface', 'Bench Height', 'Install Height',
       'Det Height'],
      dtype='object')

In [9]:
df_key_features = {
    'Blast': 'bp_id', 
    'Collar East': 'east',
    'Collar North': 'north', 
    'Collar RL': 'elev', 
    'Pre BMM RL': 'bmm_elev', 
    'Explosive Type': 'expl_type',
    'Confinement': 'confined', 
    'Bench Height': 'bp_h',
    'Inst Depth': 'inst_depth',
    'Install Height': 'inst_h',
    'Det Depth': 'det_depth', 
    'Det Height': 'det_h',
    'Direction': 'act_dir', 
    'Inclination': 'act_incl',
    '3D Dist.': 'm_3d',
    'Horiz. Dist.': 'm_h', 
    'Vert. Dist.': 'm_v', 
    }

In [10]:
df = df.rename(columns=df_key_features)
df = df[df_key_features.values()]
df.iloc[:,3:].head()

,elev,bmm_elev,expl_type,confined,bp_h,inst_depth,inst_h,det_depth,det_h,act_dir,act_incl,m_3d,m_h,m_v
0,480.097,476.597,ANFO,free face,5.097,3.5,1.597,4.711656,1.639344,207.430863,1.998595,1.214168,1.213429,0.042344
1,480.097,478.297,ANFO,free face,5.097,1.8,3.297,2.950580,3.750420,209.514424,11.388110,2.296334,2.251125,0.453420
2,479.980,476.280,ANFO,free face,4.980,3.7,1.280,4.532213,1.322787,210.199858,3.731591,0.657434,0.656040,0.042787
3,479.980,478.380,ANFO,free face,4.980,1.6,3.380,2.165292,3.823708,198.895516,12.711780,2.016424,1.967000,0.443708
4,480.261,476.561,ANFO,free face,5.261,3.7,1.561,3.996078,1.526922,199.491946,-2.276801,0.857800,0.857123,-0.034078


### Reformating blast panel names

Some names of the blast panels names from the dataframe have different formatting, which also needs to be corrected for the sake of working with clean and tidy data. 
The name of the blast panel should follow a specific format, e.g. PB1_480_006 - Pushback (phase of development) 1, Elevation, Blast panel number

Hence, a regex clause would be useful in this case

In [11]:
import re

# Function to reformat the names
def reformat_name(name):
    patterns = [r"PB(\d+)_(\d+)_BP(\d+)", r"PB(\d+)_(\d+)_(\d+)"]
    
    for pattern in patterns:
        match = re.match(pattern, name)
        if match:
            return f"PB{match.group(1)}_{match.group(2)}_{int(match.group(3)):03}"
    return name

old_names = df['bp_id'].values
formatted_names = [reformat_name(name) for name in old_names]

print(formatted_names[:5])

['PB1_480_006', 'PB1_480_006', 'PB1_480_006', 'PB1_480_006', 'PB1_480_006']


In [12]:
df['bp_id'] = formatted_names
df['bp_id'].head()

0    PB1_480_006
1    PB1_480_006
2    PB1_480_006
3    PB1_480_006
4    PB1_480_006
Name: bp_id, dtype: object

## Blast panel general data gathering

As the provided data from the software report is insufficient for analysis, additional data was gathered from the blast panel design tables to further supplement the analysis with meaningful features. In addition, these tables aim to correct some values from the report table, as some blast-related data is enetered manually and hence, prone to manual errors.

### Add date and explosive type

An additional table, containing all the general information of each panel was read and the required data was gathered and written to the blast movement dataframe. 

The additional information read from the summary table includes:
- Date
- Blast panel ID (or name)
- Explosive type used
- Total drill length (m)
- Total blasted volume (m³)
- Total blasted mass (t)
- Used explosive mass (kg)
- Overall blast powder factor (kg/m³) - Ratio between explosive mass utilized and blasted volume

In [13]:
df_summary = pd.read_excel('D:\\Downloads\\BMM ML MODEL\\dnb_summary.xlsx')
df_summary.columns = [
    'date',
    'bp_id',
    'expl_type',
    'bp_drill_m',
    'bp_vol',
    'bp_mass',
    'bp_expl_mass',
    'bp_pf'
]
df_summary.head()

,date,bp_id,expl_type,bp_drill_m,bp_vol,bp_mass,bp_expl_mass,bp_pf
0,2018-07-03,PB1_480_002,ANFO,122.9,1425.506,3221.64356,459.1,0.322061
1,2018-07-19,PB1_480_001,ANFO,504.0,6237.000,14095.62000,1982.1,0.317797
2,2018-07-26,PB1_480_003,ANFO,824.1,6522.000,14739.72000,2703.0,0.414443
3,2018-08-02,PB1_475_001,ANFO,674.4,7788.000,17600.88000,3007.0,0.386107
4,2018-08-09,PB1_475_002,ANFO,836.0,9801.000,22150.26000,3547.0,0.361902


The additionally obtained data was put into the original dataframe for each monitor observation.

In [14]:
df['date'] = np.nan
merge_cols = ['date', 'bp_id', 'expl_type']

In [15]:
df = df.merge(df_summary, on='bp_id', how='left', suffixes=('', '_update'))
df['date'] = df['date_update']
df['expl_type'] = df['expl_type_update']

# Drop the temporary columns used for update
df = df.drop(columns=['date_update', 'expl_type_update'])

In [16]:
df = df.dropna().reset_index(drop=True)
df.iloc[:,3:].head()

,elev,bmm_elev,expl_type,confined,bp_h,inst_depth,inst_h,det_depth,det_h,act_dir,act_incl,m_3d,m_h,m_v,date,bp_drill_m,bp_vol,bp_mass,bp_expl_mass,bp_pf
0,474.667,470.967,ANFO,free face,4.667,3.7,0.967,4.282504,1.061496,161.867800,5.635974,0.962202,0.957551,0.094496,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216
1,474.667,472.967,ANFO,free face,4.667,1.7,2.967,2.033786,3.229214,187.461599,7.606017,1.981058,1.963628,0.262214,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216
2,474.601,470.901,ANFO,free face,4.601,3.7,0.901,5.423375,0.376625,151.617502,-20.213188,1.517665,1.424197,-0.524375,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216
3,474.601,472.901,ANFO,free face,4.601,1.7,2.901,2.239060,3.802940,188.005486,21.072222,2.508565,2.340812,0.901940,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216
4,475.126,471.426,ANFO,free face,5.126,3.7,1.426,4.148262,1.383738,228.806465,-1.708764,1.417262,1.416632,-0.042262,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216


### Raw blast panel spreadsheets data parsing

The following code deals with looking into all relevant .xlsx files containing blast design data, useful for obtaining the additional features necessary for blast movement prediction based on a root folder location.

In [17]:
root_folder = 'D:\\Downloads\\BMM ML MODEL\\PB'

matching_files = []

for root, dirs, files in os.walk(root_folder):
    for file in files:
        # Check if file is an Excel file and contains "PB" in its name
        if (file.endswith('.xlsx') or file.endswith('.xls')) and ('PB' in file):
            matching_files.append(os.path.join(root, file))

matching_files[:5], len(matching_files)

(['D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\001\\Параметри на сондажи_PB1_410_001.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\002\\Параметри на сондажи_PB1_410_002.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\003\\Параметри на сондажи_PB1_410_003.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\004\\Параметри на сондажи_PB1_410_004.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\005\\Параметри на сондажи_PB1_410_005.xlsx'],
 271)

Two classes of files were found - those which are ready for use and those who would require manual isnpection for checking redundant files or relevance to the case study.

In [18]:
ok_files = [f for f in matching_files if (len(f) == 83 or len(f) == 82)]
problematic_files = [f for f in matching_files if len(f) != 83 and len(f) != 82]

In [19]:
two_expl_case = 'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\415\\003\\Параметри на сондажи_PB1_415_003.xlsx'
ok_files.remove(two_expl_case)
problematic_files.append(two_expl_case)

In [20]:
ok_files[:5], len(ok_files)

(['D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\001\\Параметри на сондажи_PB1_410_001.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\002\\Параметри на сондажи_PB1_410_002.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\003\\Параметри на сондажи_PB1_410_003.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\004\\Параметри на сондажи_PB1_410_004.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\410\\005\\Параметри на сондажи_PB1_410_005.xlsx'],
 204)

In [21]:
problematic_files[:5], len(problematic_files)

(['D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\415\\001 - вода и АНФО\\Параметри на сондажи_PB1_415_001.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\415\\001 - вода и АНФО\\повторно ПВР\\Параметри на сондажи_PB1_415_ВР01.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\420\\001\\алтернатива\\Параметри на сондажи_PB1_420_001_2.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\420\\005\\Параметри на сондажи_PB1_420_005_riogel.xlsx',
  'D:\\Downloads\\BMM ML MODEL\\PB\\PB 1\\ПВР\\430\\006\\Параметри на сондажи_PB1_430_006_R.xlsx'],
 67)

As it can be observed, 67 files were problematic in some way. 

Through a manual check, it was estimated that 21 were files with duplicate designs. As it was not clear for me which design were applied in the field, they were disregarded for the analysis. 

An addition 41 .xlsx files were with for miscellaneous tasks and hence were irrelevant to the analysis.

In [22]:
# TO DO
# clean problematic files and add to ok_files

### Gather additional blast panel features required for feature engineering

A function was established which read each .xlsx file of a blst design and creates a tidy dataframe ready for use.

In [23]:
def clean_panels(f_bp_df):
    '''
    Cleans blast panel excel files and returns clean DataFrame
    '''
    bp_column_names = [
    'dh_num',
    'east',
    'north',
    'elev',
    'dh_d',
    'incl',
    'burden',
    'spacing',
    'dh_len_pr',
    'expl_len_pr',
    'stem_len_pr',
    'expl_mass_pr',
    'booster_qty_pr',
    'total_expl_mass_pr',
    'ms_delay',
    'cumul_ms_delay'
    ]
    
    f_bp_df = f_bp_df.iloc[:, :16] # Get first 16 columns with necessary blast panel data
    f_bp_df.columns = bp_column_names
    f_bp_df = f_bp_df.dropna().reset_index(drop=True)
    
    return f_bp_df

The function below estimates the ideal movement direction for an arbitrary point of the blast panel, based on the drillhole delays. It is estimated as the direction of the negative gradient of the isochrones of the blast design.

In [24]:
from scipy.interpolate import griddata

def calc_ideal_dir(f_bp_df, f_loc_data):
    """
    Calculate the ideal movement direction of an arbitrary location on the blast panel.
    """
    # Extract coordinates and delay values
    x = f_bp_df['east'].values
    y = f_bp_df['north'].values
    z = f_bp_df['ms_delay'].values

    # Generate a meshgrid for the delay surface
    meshgrid_x, meshgrid_y = np.meshgrid(
        np.linspace(min(x), max(x), num=500), 
        np.linspace(min(y), max(y), num=500)
    )

    # Calculate the delay surface (z values) using interpolation
    meshgrid_z = griddata((x, y), z, (meshgrid_x, meshgrid_y), method="linear")

    # Compute spatial step sizes
    delta_x = np.abs(meshgrid_x[0, 0] - meshgrid_x[0, 1])
    delta_y = np.abs(meshgrid_y[0, 0] - meshgrid_y[1, 0])

    # Calculate the gradients of the delay surface
    gradient_x = np.gradient(meshgrid_z, axis=1) / delta_x
    gradient_y = np.gradient(meshgrid_z, axis=0) / delta_y

    # Flatten the meshgrid for gradient interpolation
    meshgrid_pts = np.array([meshgrid_x.ravel(), meshgrid_y.ravel()]).T

    # Extract the location coordinates
    loc_x, loc_y, _ = f_loc_data  # Ignoring loc_z if not needed

    # Interpolate gradients at the specified location
    loc_gradient_x = griddata(meshgrid_pts, gradient_x.ravel(), (loc_x, loc_y), method="linear")
    loc_gradient_y = griddata(meshgrid_pts, gradient_y.ravel(), (loc_x, loc_y), method="linear")

    # Normalize the gradient vector to get direction
    gradient_magnitude = np.hypot(loc_gradient_x, loc_gradient_y)
    if gradient_magnitude == 0:
        raise ValueError("Gradient magnitude is zero, unable to calculate direction.")

    loc_vector_x = loc_gradient_x / gradient_magnitude
    loc_vector_y = loc_gradient_y / gradient_magnitude

    # Calculate the azimuth angle of the ideal direction
    loc_ideal_dir_rad = np.arctan2(-loc_vector_x, -loc_vector_y)  # Angle in radians
    loc_ideal_dir_deg = (np.degrees(loc_ideal_dir_rad) + 360) % 360  # Convert to degrees

    return loc_ideal_dir_deg

The function below estiamtes the volume of each Voronoi block, corresponding to the area of effect for each drillhole and adds the estimated value to the tidied blast panel dataframe.

In [25]:
from scipy.spatial import Voronoi
from shapely.geometry import Polygon, Point

def add_voronoi_vol(f_bp_df, f_loc_data, f_loc_radius=4.75):
    '''
    Calculate Local powder factor and local charging rules based on Voronoi cells for a specified radius.
    '''
    points = f_bp_df[['east', 'north']].values
    center = f_loc_data[:2]

    # Create Voronoi diagram
    vor = Voronoi(points)
    
    circle = Point(center).buffer(f_loc_radius)
    
    # Calculate areas for Voronoi regions
    areas = []
    polygons = []
    
    for region_index in vor.point_region:
        vertices = vor.regions[region_index]
        if -1 in vertices or len(vertices) == 0:  # Skip infinite or empty regions
            areas.append(np.nan)
            polygons.append(None)
            continue
    
        # Create the polygon for the region
        region_polygon = Polygon([vor.vertices[i] for i in vertices if i >= 0])
        
        # Check if polygon centroid is within the circle
        if circle.contains(region_polygon.centroid):
            areas.append(region_polygon.area)
            polygons.append(region_polygon)
        else:
            areas.append(0)
            polygons.append(None)
    
    f_bp_df['voronoi_vol'] = -1.0
    visited = []
    
    for poly in polygons:
        if poly is None:  # Skip None polygons
            continue
        
        for idx, row in f_bp_df.iterrows():
            point = Point(row['east'], row['north'])
            
            if idx not in visited and poly.contains(point):
                f_bp_df.at[idx, 'voronoi_vol'] = poly.area * row['bp_h']
                visited.append(idx)

    return f_bp_df

The function below estiamtes the type of booster used for initiating the the explosive mass in the drillhole.

In [26]:
def get_booster_type(f_bp_df, loc_data, min_index):
    loc_x, loc_y, loc_z = loc_data        
    
    booster = f_bp_df['total_expl_mass_pr'].values[min_index] - f_bp_df['expl_mass_pr'].values[min_index]
    return booster

As two types of explosives are used, instead of using a categorical value, the nominal amount of gas products was applied to estimate not the powder factor for each blasthole, but the amount of local gas products. Hence, the engineered feature was named *Local gas products factor*.

In [27]:
expl_gas_prod_dict = {
    'ANFO': 988,
    'Riogel': 933
}

The additional features which were aimed to be extracted for predictive model-building include:
- Ideal movement direction (°)
- Voronoi rock volume for each drillhole (m³)
- Local drillhole length sum (m)
- Local explosive charghe length sum (m)
- Local stemming length sum (m)
- Closes booster type
- Distance to first blasted drillhole (m)
- Direction angle to first blasted drillhole from monitoring location (°)

In [28]:
df['ideal_dir'] = np.nan
df['voronoi_rock_vol'] = np.nan
df['loc_dh_len'] = np.nan
df['loc_expl_len'] = np.nan
df['loc_stem_len'] = np.nan
df['loc_expl_mass'] = np.nan
df['booster'] = np.nan
df['dist_first'] = np.nan
df['loc_to_first_dir'] = np.nan

df.iloc[:,13:].head()

,act_incl,m_3d,m_h,m_v,date,bp_drill_m,bp_vol,bp_mass,bp_expl_mass,bp_pf,ideal_dir,voronoi_rock_vol,loc_dh_len,loc_expl_len,loc_stem_len,loc_expl_mass,booster,dist_first,loc_to_first_dir
0,5.635974,0.962202,0.957551,0.094496,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7.606017,1.981058,1.963628,0.262214,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,-20.213188,1.517665,1.424197,-0.524375,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,21.072222,2.508565,2.340812,0.901940,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,-1.708764,1.417262,1.416632,-0.042262,2018-08-23,926.7,12083.0,27307.58,3845.0,0.318216,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The code below iterates through all ready-to-use files and adds the forementioned features to the blast movement monitors' dataset.

In [29]:
import time
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

loop_start_time = time.time()

for f in ok_files[:]:    
    start_time = time.time()
    bp_id = f[67:78]
    
    cur_bp_df = pd.read_excel(f)
    cur_bp_df = clean_panels(cur_bp_df).reset_index(drop=True)

    target_elevation = int(bp_id[4:7])
    cur_bp_df['bp_h'] = cur_bp_df['elev'] - target_elevation

    if cur_bp_df.empty:
        continue
    else:    
        matching_indices = df.index[df['bp_id'] == bp_id].tolist()  # Find matching indices
    
        if matching_indices:
            print(bp_id + ' started')
            for i in matching_indices:
                # Extract location data
                loc_data = df.iloc[i, 1:4].values
                loc_east, loc_north, loc_elev = loc_data
        
                # Calculate distances and find the closest blasthole to current monitoring locaton in current blast panel
                delta_x = cur_bp_df['east'].values - loc_east
                delta_y = cur_bp_df['north'].values - loc_north
                
                distances = (delta_x**2 + delta_y**2)**0.5
                min_dist_index = np.argmin(distances)
                first_dh_index = cur_bp_df.index[cur_bp_df['ms_delay'] == 0].tolist()
                filtered_distance = distances[first_dh_index]

                # Find relative position of first drillhole to monitoring location
                loc_to_first_dir = np.degrees(np.arctan2(delta_y[first_dh_index][0], delta_x[first_dh_index][0]))
        
                # Get explosive density based on explosive type for closest blasthole to current monitoring location
                expl_gas_prod = expl_gas_prod_dict[df.loc[i, 'expl_type']]
                new_cur_bp_df = add_voronoi_vol(cur_bp_df, loc_data,  4.75)
        
                # Update columns with calculations
                df.loc[i, 'ideal_dir'] = calc_ideal_dir(new_cur_bp_df, loc_data)
                df.loc[i, 'voronoi_rock_vol'] = new_cur_bp_df.loc[new_cur_bp_df['voronoi_vol']>0, 'voronoi_vol'].sum()
                df.loc[i, 'loc_dh_len'] = new_cur_bp_df.loc[new_cur_bp_df['voronoi_vol']>0, 'dh_len_pr'].sum()
                df.loc[i, 'loc_expl_len'] = new_cur_bp_df.loc[new_cur_bp_df['voronoi_vol']>0, 'expl_len_pr'].sum()
                df.loc[i, 'loc_stem_len'] = new_cur_bp_df.loc[new_cur_bp_df['voronoi_vol']>0, 'stem_len_pr'].sum()
                df.loc[i, 'loc_expl_mass'] = new_cur_bp_df.loc[new_cur_bp_df['voronoi_vol']>0, 'expl_mass_pr'].sum()
                df.loc[i, 'booster'] = get_booster_type(new_cur_bp_df, loc_data, min_dist_index)
                df.loc[i,'dist_first'] = filtered_distance
                df.loc[i, 'loc_to_first_dir'] = loc_to_first_dir
    
            end_time = time.time()
            calc_time = end_time - start_time
            print(bp_id + ' finished: ' + str(round(calc_time, 1))+'s')

loop_end_time = time.time()

PB1_410_001 started
PB1_410_001 finished: 12.4s
PB1_410_002 started
PB1_410_002 finished: 23.6s
PB1_410_003 started
PB1_410_003 finished: 28.7s
PB1_410_004 started
PB1_410_004 finished: 23.2s
PB1_410_005 started
PB1_410_005 finished: 30.8s
PB1_415_002 started
PB1_415_002 finished: 10.6s
PB1_415_004 started
PB1_415_004 finished: 60.0s
PB1_415_005 started
PB1_415_005 finished: 26.6s
PB1_415_006 started
PB1_415_006 finished: 51.7s
PB1_415_007 started
PB1_415_007 finished: 36.3s
PB1_415_008 started
PB1_415_008 finished: 28.9s
PB1_415_009 started
PB1_415_009 finished: 16.3s
PB1_415_010 started
PB1_415_010 finished: 8.3s
PB1_420_001 started
PB1_420_001 finished: 24.6s
PB1_420_003 started
PB1_420_003 finished: 21.4s
PB1_420_004 started
PB1_420_004 finished: 23.0s
PB1_420_006 started
PB1_420_006 finished: 17.5s
PB1_420_007 started
PB1_420_007 finished: 34.4s
PB1_420_008 started
PB1_420_008 finished: 13.8s
PB1_420_009 started
PB1_420_009 finished: 30.8s
PB1_420_010 started
PB1_420_010 finished:

In [30]:
print(f"Total elapsed time: {(loop_end_time - loop_start_time):.2f}s")

Total elapsed time: 5472.53s


As it can be observed, the process took a bit of time, but it definitely saved lots of manual work!
Now, we are ready to save the file, check its new contents some anomalous behavior and start analyzing!

In [31]:
df.to_excel('obtained_data.xlsx')

**Vol. 2.** continues with analyzing the data and using machine learning models to attempt and model the blast movement phenomenon.